# 13 - Data Path

## Introduction

The **Data Path** connects all the components we've built:
- Program Counter
- Memory (RAM)
- Register File
- ALU
- Instruction Register

It routes data between components based on control signals.

## Learning Objectives

1. Understand how components connect
2. Learn about data routing with multiplexers
3. Build the complete Data Path

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))
sys.path.insert(0, str(project_root))

from computer.counters import ProgramCounter
from computer.memory import RAM
from computer.registers import RegisterFile
from computer.alu import ALU
from computer.clock import ControlSignals
from computer import int_to_bits, bits_to_int
from typing import List, Dict
print("Setup complete!")

## Data Path Architecture

```mermaid
flowchart TB
    PC[PC] --> MEM[Memory/RAM]
    MEM --> IR[Instruction Register]
    IR --> DEC[Decoder]
    DEC --> CU[Control Unit]
    
    MEM <--> DataBus[Data Bus]
    RF[Register File] <--> DataBus
    ALU[ALU] --> DataBus
    RF --> ALU
    
    PC --> |+1/Load| PC
    DataBus --> |Result| RF
```

## Exercise 1: Data Path

In [ ]:
class DataPath:
    """CPU Data Path - connects all components."""
    
    def __init__(self):
        self.pc = ProgramCounter()
        self.memory = RAM()
        self.reg_file = RegisterFile()
        self.alu = ALU()
        self.ir = [0] * 16  # Instruction Register
        self.flags = {'Z': 0, 'C': 0, 'N': 0, 'V': 0}
    
    def execute_cycle(self, signals: ControlSignals, decoded: Dict) -> None:
        """
        Execute one cycle based on control signals.
        
        Args:
            signals: Control signals from Control Unit
            decoded: Decoded instruction fields
        """
        # YOUR CODE HERE
        # Based on signals, route data between components:
        # - Read registers if needed
        # - Perform ALU operation if needed
        # - Read/write memory if needed
        # - Write result to register if needed
        # - Update PC (load, increment, or hold)
        pass
    
    def fetch_instruction(self) -> List[int]:
        """
        Fetch instruction from memory at current PC.
        
        Returns:
            16-bit instruction
        """
        # YOUR CODE HERE
        # Read two bytes from memory (16-bit instruction)
        pass
    
    def load_instruction(self, instruction: List[int]) -> None:
        """Load instruction into IR."""
        self.ir = instruction.copy()
    
    def get_pc(self) -> List[int]:
        """Get current PC value."""
        return self.pc.read()
    
    def set_pc(self, value: List[int]) -> None:
        """Set PC to new value."""
        self.pc.clock(load=1, load_value=value, increment=0, reset=0, clk=1)

# Test
dp = DataPath()

# Load some values into registers
addr_r1 = [1, 0, 0]  # R1
addr_r2 = [0, 1, 0]  # R2
dp.reg_file.write(addr_r1, int_to_bits(10, 8), 1, 0)
dp.reg_file.write(addr_r1, int_to_bits(10, 8), 1, 1)
dp.reg_file.write(addr_r2, int_to_bits(20, 8), 1, 0)
dp.reg_file.write(addr_r2, int_to_bits(20, 8), 1, 1)

print(f"R1 = {bits_to_int(dp.reg_file.read(addr_r1))}")
print(f"R2 = {bits_to_int(dp.reg_file.read(addr_r2))}")
print(f"PC = {bits_to_int(dp.get_pc())}")

## Data Path Execution Examples

In [ ]:
# Simulate ADD R0, R1, R2
dp = DataPath()

# Set up registers
addr_r0 = [0, 0, 0]
addr_r1 = [1, 0, 0]
addr_r2 = [0, 1, 0]
dp.reg_file.write(addr_r1, int_to_bits(15, 8), 1, 0)
dp.reg_file.write(addr_r1, int_to_bits(15, 8), 1, 1)
dp.reg_file.write(addr_r2, int_to_bits(27, 8), 1, 0)
dp.reg_file.write(addr_r2, int_to_bits(27, 8), 1, 1)

print("Before ADD:")
print(f"  R0 = {bits_to_int(dp.reg_file.read(addr_r0))}")
print(f"  R1 = {bits_to_int(dp.reg_file.read(addr_r1))}")
print(f"  R2 = {bits_to_int(dp.reg_file.read(addr_r2))}")

# Simulate ADD execution
signals = ControlSignals()
signals.reg_write = 1
signals.alu_op = [0, 0, 0, 0]  # ADD

decoded = {
    'opcode_name': 'ADD',
    'rd': 0, 'rs1': 1, 'rs2_imm': 2,
    'rd_bits': [0, 0, 0],
    'rs1_bits': [1, 0, 0],
    'rs2_bits': [0, 1, 0],
}

dp.execute_cycle(signals, decoded)

print("\nAfter ADD R0, R1, R2:")
print(f"  R0 = {bits_to_int(dp.reg_file.read(addr_r0))} (should be 42)")

## Copy Your Implementation

Once your code works, copy the `DataPath` class to `src/computer/datapath.py`

## Validation

In [ ]:
from utils.checker import check
check('datapath')

## Summary

The Data Path connects:

| Component | Role |
|-----------|------|
| PC | Tracks current instruction |
| Memory | Stores program and data |
| Register File | Fast operand storage |
| ALU | Performs operations |
| IR | Holds current instruction |

### What's Next?

The **CPU** combines the Data Path with the Control Unit to create the complete fetch-decode-execute cycle!